# Testing Mantid's CylindricalAbsorption wrapper
<div class="alert alert-block alert-info">
<b>Information:</b> This notebook shows how to use Mantid's CylindricalAbsorption algorithm with a `scipp` Dataset.

<b>Requirements:</b> To run this notebook, you need `mantid`, `scipp` and `numpy` installed as well as the Python script `absorption.py` placed in the same folder as this notebook.   
Additionally, the test file `wish_test.nxs` should reside in the same directory.
</div>

In [ ]:
import numpy as np
import scipp as sc
import mantid.simpleapi as simpleapi
from scipp.compat.mantid import load
from absorption import absorption_correction


## Loading data

Load a suitable dataset first. This is a **HUGE** file so each operation will be quite slow.
The load operation takes ~8s on the DMSC workstation.

In [ ]:
filename = "wish_test.nxs"
dataset = sc.compat.mantid.load(filename=filename)

The file is loaded as a `DataArray` with following sizes:

In [ ]:
print(dataset.dims)
print(dataset.shape)

## Defining absorption parameters

Our absorption should be calculated for the sample.

In [ ]:
scatter_from = "Sample"

## Calculating absorption

Now we are ready to calculate the correction, passing the sample definition and example cylinder dimensions to the calculator.
We need these, since the data file contains no sample definition.
This calculation takes some time.

In [ ]:
correction = absorption_correction(dataset, filename,
                                   ScatterFrom=scatter_from,
                                   CylinderSampleHeight=3,
                                   CylinderSampleRadius=10)

The correction dataset uses wavelengths units, so we need to convert it back to time-of-flight, as in the original sample.

In [ ]:
correction = sc.neutron.convert(correction, 'wavelength', 'tof')

The absorption correction has been calculated to match the input dataset over bins ranging from

In [ ]:
correction.coords['spectrum'].values[0]

to

In [ ]:
correction.coords['spectrum'].values[-1]

## More complex parameter definition

We may also define other parameters required for Mantid's algorithm.

In [ ]:
sample_number_density = 0.07192
number_wavelength_points = 5
cylinder_sample_height = 4
cylinder_sample_radius = 0.4
number_of_slices = 2

These parameters can just be appended to the `absorption_correction` call.

In [ ]:
correction = absorption_correction(dataset, filename,
                                   ScatterFrom=scatter_from,
                                   SampleNumberDensity=sample_number_density,
                                   NumberOfWavelengthPoints=number_wavelength_points,
                                   CylinderSampleHeight=cylinder_sample_height,
                                   CylinderSampleRadius=cylinder_sample_radius,
                                   NumberOfSlices=number_of_slices)

In [ ]:
correction = sc.neutron.convert(correction, "Wavelength", "Tof")
print(correction.coords['spectrum'].values[0])
print(correction.coords['spectrum'].values[-1])

Finally, we can apply the correction to the initial data array.

In [ ]:
corrected = dataset/correction

Adventurous users might want to visualize the correction itself.

In [ ]:
from scipp.plot import plot

In [ ]:
sc.show(correction)